In [1]:
# TODO: filter by date?; check data quality (if really from city)
# TODO: filter out retweets and potential spam
# in the London data set until April 2019 are tweets from Rotterdam and Amsterdam, so you can use country_code = 'GB' to filter the tweets.

In [2]:
import sqlite3
import pandas as pd
from sqlite3 import Error
root = '/Volumes/Festplatte/data-UA/'
dbfile = 'NYC/2018/month_2018_08.db'
president = 'trump'

In [3]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)

    return conn

In [4]:
conn = create_connection(root + dbfile)

In [5]:
def db_select(conn, table, limit, select = '*', where = 'True'):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    cur.execute(f'SELECT {select} FROM {table} WHERE {where} LIMIT {limit}')

    rows = cur.fetchall()

    for row in rows:
        print(row)

In [6]:
db_select(conn, 'sentiment', limit='10')

(1, 0.0, 0.0, 0.0, 1.0)
(2, 0.9623, 0.448, 0.0, 0.552)
(3, 0.0, 0.0, 0.0, 1.0)
(4, 0.6133, 0.255, 0.091, 0.654)
(5, 0.7269, 0.433, 0.0, 0.567)
(6, 0.0772, 0.075, 0.0, 0.925)
(7, 0.0, 0.0, 0.0, 1.0)
(8, 0.594, 0.243, 0.0, 0.757)
(9, 0.0, 0.0, 0.0, 1.0)
(10, -0.0622, 0.215, 0.182, 0.602)


In [7]:
table_list = [a for a in conn.execute("SELECT name FROM sqlite_master WHERE type = 'table'")]
print('The following tables exist: ', table_list)

The following tables exist:  [('hashtags',), ('media',), ('place',), ('tweets',), ('urls',), ('user',), ('user_full',), ('user_mentions_name',), ('sentiment',), ('users_2020_0409_plus3',), ('LIWC',)]


In [8]:
# # Create a SQL connection to our SQLite database
# conn = sqlite3.connect(root + dbfile)

# # creating cursor
# cur = conn.cursor()

# # reading all table names
# table_list = [a for a in cur.execute("SELECT name FROM sqlite_master WHERE type = 'table'")]
# # here is you table list
# print(table_list)

In [9]:
def get_col_names():
    """ get all column names of an SQLite database
    :param db_file: database file
    :return: Tuple of two Lists
    """
    num_fields = len(cur.description)
    field_names = [i[0] for i in cur.description]
    return (field_names, num_fields)

In [10]:
def get_row_count(tablename):
    """ get the total count of rows in an SQLite database
    :param tablename: String
    :return: Integer
    """
    cur = conn.cursor()
    cur.execute(f'select * from {tablename}')
    results = cur.fetchall()
    row_count = len(results)
    # print(f'{row_count} rows in {tablename} database from {dbfile}')
    return row_count

In [11]:
get_row_count('sentiment')

1235925 rows in sentiment database from NYC/2018/month_2018_08.db


### Create Dataset

In [12]:
# join sentiment and tweet table
# only select english tweets
data_en_keywords = pd.read_sql_query(
    f"""
    SELECT * 
    FROM tweets 
    INNER JOIN sentiment 
    USING (item_number) 
    WHERE (
        lang == 'en'
        AND
        text LIKE '%{president}%'
    );
    """, 
    conn)
# alternatively: # cur.execute('SQL STATEMENT")

In [13]:
# DONT NEED TO FILTER BY KEYWORD BECAUSE SQL DOES IT FOR ME ('trump" elemof "DonaldTrump")
# # filter out relevant tweets: rule-based approach
# # tags, filter by word
# data_en_keywords = data_en.loc[
#     data_en['text'].isin(keywords)
# ]

In [14]:
# PYTHON IS CASE-SENSITIVE - SQL IS NOT HERE
# data_en[data_en['text'].str.contains("trump")].shape[0] + data_en[data_en['text'].str.contains("Trump")].shape[0]

In [15]:
# now the compounded sentiment value should be in the cursor
assert 'Compound' in data_en_keywords.columns

In [16]:
# keywords = ('Trump', 'DonaldTrump', 'RealDonaldTrump', 'BorisJohnson', 'BoJo', '#Trump', '#BorisJohnson')
# hashtags = pd.read_sql_query(f"SELECT DISTINCT item_number FROM hashtags WHERE text IN {keywords}", conn)
# TODO: check which way to do it: with keywords or like %%!
hashtags = pd.read_sql_query(f"SELECT DISTINCT item_number FROM hashtags WHERE text LIKE '%{president}%';", conn)

In [17]:
# posts that are not present in the current data_en_keywords that has a relevant hashtag is now added to the data
newly_relevant_hashtags = list(set(hashtags['item_number']) - set(data_en_keywords['item_number']))
print(f'{len(newly_relevant_hashtags)} posts are relevant because of hashtags, which were not present in the dataset before.')

336 posts are relevant because of hashtags, which were not present in the dataset before.


In [18]:
tweets_by_hashtags = pd.read_sql_query(f"""
                                        SELECT * 
                                        FROM tweets INNER JOIN sentiment 
                                        USING (item_number) 
                                        WHERE (
                                            lang == 'en' 
                                            AND 
                                            item_number IN {tuple(newly_relevant_hashtags)}
                                        )
""", conn)

In [19]:
print(f'There are {tweets_by_hashtags.shape[0]} tweets which are in english and are relevant by using the hashtag')

There are 0 tweets which are in english and are relevant by using the hashtag


In [20]:
data_en_keywords_hashtags = pd.concat([data_en_keywords, tweets_by_hashtags], axis=0)

In [21]:
# TODO: filter out retweets and potential spam | tokenization, lemmatization and stopword removal

In [22]:
# TODO: vectorization (bag of words or bag-of-ngrams or word2vec)

In [29]:
import os
cities = ['Birmingham', 'LA', 'London', 'NYC']
data_path = root + 'data/'
os.mkdir(data_path)
for city in cities:
    os.mkdir(data_path + city)

In [23]:
filepath = data_path + 'data_en_keywords_hashtags.pkl'
# TODO: update it when I automate to generate dataset for every month
data_en_keywords.to_pickle(filepath)
# data_en_keywords = pd.read_pickle(filepath)

In [24]:
print(f'The shape of the preprocessed data is: {data_en_keywords_hashtags.shape}')

The shape of the preprocessed data is: (24232, 17)


In [25]:
# close the connection
conn.close()

## Looping over every database

In [33]:
years = [str(year) for year in range(2018, 2023)]
print(years)

['2018', '2019', '2020', '2021', '2022']


In [46]:
def get_files(dir):
    """ get all files from a directory 
    that do not start with a '.'
    :param dir: String
    :return: List of Strings
    """
    return [s for s in os.listdir(dir) if not s.startswith('.')]

In [48]:
get_files(root + cities[0] + "/" + years[2])

['2018', '2019', '2020', '2021', '2022']